# Projet 6 : Catégorisez automatiquement des questions
# <u>B. Topic Modeling</u> <br/>

In [72]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob

from bs4 import BeautifulSoup
import unicodedata
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize.toktok import ToktokTokenizer
from contractions import CONTRACTION_MAP
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from collections import Counter

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn import model_selection, metrics
from sklearn.decomposition import NMF, LatentDirichletAllocation

%matplotlib inline

# 1. Chargement des données pré-traitées

Nos données sont réparties dans 5 fichiers représentant une taille totale de 0,12Go.

In [76]:
df = pd.read_csv('cleaned_data.csv')
#replace NaN by empty string
df = df.replace(np.nan, '', regex=True)

In [77]:
df.shape

(64432, 7)

In [78]:
df.head()

,TITLE,BODY,SCORE,TAGS,TITLE_P,BODY_P,TAGS_P
0,Java generics variable <T> value,<p>At the moment I am using the following code...,6,<java><generics>,java gener variabl valu,moment use follow code filter jpa reduc block ...,java generics
1,How a value typed variable is copied when it i...,<blockquote>\n <p>Swift's string type is a va...,6,<swift><function><value-type>,valu type variabl copi pass function hold copi,swift string type valu type creat new string v...,swift function value-type
2,Error while waiting for device: The emulator p...,<p>I am a freshman for the development of the ...,6,<android><android-studio><android-emulator><avd>,error wait devic emul process avd kill,freshman develop andriod suffer odd question r...,android android-studio android-emulator avd
3,gulp-inject not working with gulp-watch,<p>I am using gulp-inject to auto add SASS imp...,10,<javascript><node.js><npm><gulp><gulp-watch>,gulp inject work gulp watch,use gulp inject auto add sass import newli cre...,javascript node.js npm gulp gulp-watch
4,React - Call function on props change,<p>My TranslationDetail component is passed an...,12,<reactjs><react-router>,react call function prop chang,translationdetail compon pass id upon open bas...,reactjs react-router


# 2. Transformation des données

## 2.1 Echantillonage

Travaillons sur un échantillon de 15 000 posts.

In [122]:
df_sample = df.sample(10000)

In [80]:
df_sample.shape

(10000, 7)

## 2.2 Filtre sur les tags les plus fréquents

Pour chaque tag on stocke son nombre d'occurences.

In [81]:
counts = Counter()
for sentence in df['TAGS_P']:
    counts.update(sentence.split())
tags_df = pd.DataFrame.from_dict(counts, orient='index')
tags_df.reset_index(drop = False, inplace = True)
tags_df= tags_df.rename(columns={'index':'tag', 0:'count'})

La structures tags_df contient pour chacun des tags son occurence. <br/>
Gardons que les tags qui sont présents dans au moins 150 documents.

In [123]:
len(tags_df[tags_df['count'] > 150])

175

Nous nous retrouvons donc avec 175 tags au lieu des 14000 dans le dataset original.

=> Filtrons maintenant notre dataset sample en ne gardant que les posts dont les tags sont les plus fréquents.

In [124]:
frequent_tags = tags_df[tags_df['count'] > 150]['tag'].tolist()

In [125]:
df_sample['TAGS_P'] = df_sample['TAGS_P'].apply(lambda x: ' '.join( [w for w in x.split() if w in frequent_tags] ))
df_sample['TAGS_P'].replace('', np.nan, inplace=True)
df_sample = df_sample.dropna()

In [126]:
df_sample.shape

(9290, 7)

Notre dataset contient maintenant uniquement les posts avec tags fréquents.

## 2.3 Découpage en jeu entrainement et test

In [127]:
X = df_sample[['TITLE_P', 'BODY_P']]
target_tags = pd.DataFrame(df_sample['TAGS_P'].apply(lambda x: [t for t in x.split()]))

In [128]:
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(target_tags['TAGS_P'])

In [129]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(X,Y,test_size = 0.3,random_state = 0, shuffle = True)

In [130]:
print("train", x_train.shape)
print("test ",x_test.shape)

train (6503, 2)
test  (2787, 2)


On concatène le titre et le body qui ont été pré-traités pour notre apprentissage.

In [131]:
x_train_text = x_train['TITLE_P'] + ' ' + x_train['BODY_P']
x_test_text = x_test['TITLE_P'] + ' ' + x_test['BODY_P']

## 2.4 Tokenization du text

Pour pouvoir appliquer nos algorithmes de machine learning à nos données textuelles, il faut en extraire les features et représenter notre texte dans un modèle "mathématique".
Pour celà nous allons utiliser la modélisation **Bag of Words** qui va donner une représentation sous forme de matrice de nos données.

- min_df = 5 : le mot doit être présent au moins dans 5 documents
- max_df = 95% : si le mot est présent dans plus de 95% des documents, il ne nous aidera pas à trouver différencier les documents et on le supprime
- max_features : on se limite à 300 mots maximum (nous avons vu plus haut que 598 mots sont présents au moins dans 500 posts).

In [132]:
cv = CountVectorizer(min_df=5, max_df=0.95, max_features=600)
X_train_counts = cv.fit_transform(x_train_text)
X_train_counts.shape

(6503, 600)

In [133]:
X_test_counts = cv.transform(x_test_text)
X_test_counts.shape

(2787, 600)

## 2.5 From occurrences to frequencie

In [134]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(6503, 600)

In [135]:
X_test_tfidf = tfidf_transformer.transform(X_test_counts)
X_test_tfidf.shape

(2787, 600)

# 3. Modélisation avec méthodes non supervisées

In [136]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier().fit(X_train_tfidf,y_train)

In [137]:
clf.score(X_test_tfidf, y_test)

0.14711158952278436

In [54]:
y_test.shape

(27, 41)